### An example of Automatic Differentiation

Pauli propagation has promise in optimizing variational models because its elementary operations are very simple and automatically differentiable. Additionally, it may be the case that strongly truncated simulations still yield final parameters that translate to high-quality quantum circuits on, for example, a quantum computer. This means that we could (perhaps) very cheaply train/optimize, and then deploy the learned model on a different classical or quantum method. There are some initial indications of this being possible, but they require further exploration. 

In this notebook, we demonstrate how to calculate fast automatic gradients of a quantum circuit using Pauli propagation.

In [1]:
using PauliPropagation

Let us set up a simulation:

In [2]:
nq = 16

topology = bricklayertopology(nq);

We define a transverse field Hamiltonian, whose derivative we will compute. This could be used within a variational energy minimization routine to find its ground state. 

The Hamiltonian here reads $H = \sum_{i}X_i + \sum_{\langle i, j\rangle}Z_iZ_j$ where $ \langle i, j\rangle$ denotes neighbors on the topology.

In [3]:
H = PauliSum(nq)

for qind in 1:nq
    add!(H, :X, qind, 1.0)
end

for pair in topology
    add!(H, [:Z, :Z], collect(pair), 1.0)
end

H

PauliSum(nqubits: 16, 31 Pauli terms:
 1.0 * IIXIIIIIIIIIIIII
 1.0 * IIIIIIIIIIIIXIII
 1.0 * IIIIIIIIIIIIIIIX
 1.0 * IZZIIIIIIIIIIIII
 1.0 * IIIIIIIIIIIIIXII
 1.0 * IIIIIIXIIIIIIIII
 1.0 * IIIIIIIIZZIIIIII
 1.0 * XIIIIIIIIIIIIIII
 1.0 * IIIIXIIIIIIIIIII
 1.0 * IIIIIIIIIIIIZZII
 1.0 * IIIIZZIIIIIIIIII
 1.0 * IIIIIZZIIIIIIIII
 1.0 * IIIIIIIZZIIIIIII
 1.0 * IIIIIXIIIIIIIIII
 1.0 * IIIIIIIIIIIZZIII
 1.0 * IIIIIIIIIIXIIIII
 1.0 * IIIIIIIIIZZIIIII
 1.0 * IIIZZIIIIIIIIIII
 1.0 * IIIIIIIIIIIXIIII
 1.0 * IIIXIIIIIIIIIIII
  ⋮)

Define some generic quantum circuit

In [4]:
nl = 4

circuit = hardwareefficientcircuit(nq, nl; topology=topology)
nparams = countparameters(circuit)

252

Importantly, we need to set our truncations. Depending on which package and which method you are using to compute your gradients, you can use different truncations. 

`ReverseDiff` for example is a sophisticated package for automatic _reverse-mode_ differentiation (others may or may not work better). It will build a computational graph that it then differentiates using the chain rule. This is how large-scale neural networks are trained, and is commonly referred to as gradient _backpropagation_. The challenge here is that the fastest gradients are achieved with a _compiled_ version of the gradient function where the graph for the chain rule is computed once (to the best of our knowledge). In that case, only truncations during the initial computation will be respected. Truncations that we think work well here are `max_weight`, `max_freq`, and `max_sins`, as they do not depend on the particular parameters of the quantum circuit. On the other hand, which paths are explore with truncations such as `min_abs_coeff` will not be updated (again, to the best of our knowledge) as the gradients are computed. If you want to use such parameter-dependent truncations, you may need to use un-compiled gradient functions.

Packages such as `ForwardDiff` or manual numerical differentiation, on the other hand, always involve computation of the loss function, which is affected by all truncations. Unfortunately, these methods are slower for circuits with more than several dozen parameters.

Generate some generic parameters:

In [5]:
using Random
Random.seed!(42)
thetas = randn(nparams);

One expectation evaluation

In [6]:
max_freq = 25
max_weight = 5

@time psum = propagate(circuit, H, thetas; max_freq, max_weight);
overlapwithzero(psum)

  0.833596 seconds (1.08 M allocations: 58.479 MiB, 85.88% compilation time)


-1.8904666076774599

**Note**: To make these following functionsfaster, you should look into `let` blocks for local variable namespaces. You will want to define _closures_ and refrain from using global variables.

#### What does not work:

Now wrap these steps into a function that takes only `thetas` as argument. (keep in mind our comment about using `let` blocks)

This loss function does not work because the `ReverseDiff` package needs to propagate its custom coefficient type. But `H` is already strictly typed. So the following loss function would not be automatically differentiable.

In [7]:
function naivelossfunction(thetas)   
    psum = propagate(circuit, H, thetas; max_freq, max_weight);
    return overlapwithzero(psum)
end

naivelossfunction (generic function with 1 method)

The loss works

In [8]:
@time naivelossfunction(thetas)

  0.271859 seconds (1.22 M allocations: 67.234 MiB, 3.01% gc time, 57.10% compilation time)


-1.8904666076774599

But the gradient would break with a cryptic error message.

In [9]:
# using Pkg; Pkg.add("ReverseDiff")
using ReverseDiff: gradient

## this errors 
# gradient(naivelossfunction, thetas)

#### What works:

To create a loss function that indeed works, we build the Hamiltonian with the correct coefficient type within the loss function. What tends to work is to type the coefficients in the Pauli sum to the element type of `thetas`, but this will not always work. In this case, it will make everything differentiable.

Another somewhat annoying thing we need to do is to manually convert the coefficients of the Hamiltonian into `PauliFreqTracker` objects. Normally, in `propagate()` we automatically convert coefficients to our `PauliFreqTracker` types when `max_freq` or `max_sin` are set. In the in-place version `propagate!()`, we intentionally do not convert automatically. But here we need to use the `propagate!()` function because otherwise the ReverseDiff library errors because we copy the Pauli sum while that library is building its differentiation graph. Annoying... we agree! We will keep an eye out for better options.

In [10]:
function lossfunction(thetas)
    # differentiation libraries use custom types to trace through the computation
    # we need to make all of our objects typed like that so that nothing breaks
    CoeffType = eltype(thetas)

    # define H again 
    H = PauliSum(nq, CoeffType)
    for qind in 1:nq
        add!(H, :X, qind, CoeffType(1.0))
    end
    for pair in topology
        add!(H, [:Z, :Z], collect(pair), CoeffType(1.0))
    end

    # wrapp the coefficients into PauliFreqTracker so that we can use `max_freq` truncation.
    # usually this happens automatically but the in-place propagate!() function does not allow that.
    wrapped_H = wrapcoefficients(H, PauliFreqTracker)

    # be also need to run the in-place version with `!`, because by default we copy the Pauli sum
    output_H = propagate!(circuit, wrapped_H, thetas; max_freq, max_weight);
    return overlapwithzero(output_H)
end

lossfunction (generic function with 1 method)

Instead, we need to define a loss function that creates H every time with the correct coefficient type:

In [11]:
@time lossfunction(thetas)

  0.233968 seconds (438.39 k allocations: 27.163 MiB, 50.53% compilation time)


-1.8904666076774599

And gradients work.

In [12]:
@time gradient(lossfunction, thetas)

  3.714606 seconds (21.02 M allocations: 931.987 MiB, 8.48% gc time, 55.39% compilation time)


252-element Vector{Float64}:
 -0.5057064083446595
  0.025028009253606664
 -0.47337187596655284
  0.09285919084901251
  0.0900343739787604
  0.26751480771265024
  0.07722722421907667
  0.5186194278104864
 -0.4367624010611486
  0.7767471716936789
  0.0227644087188041
  0.908680529040292
 -0.11929346203144106
  ⋮
 -0.4835428738051231
 -0.2487625833982674
 -0.7705492109786312
  0.2651823350777113
 -1.2311240255419973
  1.4595405277023366
  0.3972920774850484
 -0.7513152897757447
 -1.0433085832333235
  0.6822884973623046
 -0.4331718877680776
 -0.24910861064112663

Now import ReverseDiff and follow their example for very fast gradients:

In [13]:
using ReverseDiff: GradientTape, gradient!, compile

In [14]:
### This is following an ReverseDiff.jl example

# some inputs and work buffer to play around with
grad_array = similar(thetas);

# pre-record a GradientTape for `gradsimulation` using inputs of length m with Float64 elements
@time const simulation_tape = GradientTape(lossfunction, thetas)

# first evaluation compiles and is slower
@time gradient!(grad_array, simulation_tape, thetas)
# second evaluation
@time gradient!(grad_array, simulation_tape, thetas);

  1.420120 seconds (17.09 M allocations: 739.430 MiB, 23.65% gc time, 0.53% compilation time)
  0.696674 seconds (238.88 k allocations: 11.783 MiB, 25.98% compilation time)
  0.522534 seconds


In [15]:
# compile to make it even faster
@time const compiled_simulation_tape = compile(simulation_tape)

# some inputs and work buffer to play around with
grad_array_compiled = similar(thetas);

# first evaluation compiles and is slower
@time gradient!(grad_array_compiled, compiled_simulation_tape, thetas)
# second evaluation
@time gradient!(grad_array_compiled, compiled_simulation_tape, thetas);

  3.427378 seconds (27.74 M allocations: 1.155 GiB, 7.41% gc time, 13.28% compilation time)
  0.381263 seconds (74.84 k allocations: 3.675 MiB, 24.42% compilation time)
  0.285837 seconds


`grad_array` here carries the gradient result. It is changed in-place in `gradient!` so that the array does not need to get allocated over and over. And the results between the compiled version and the non-compiled version is the same.

In [16]:
grad_array ≈ grad_array_compiled

true

See how calculating the gradient is only a few times slower than calculating the loss! The magic if reverse-mode differentiation. Feel free to explore other automatic differentiation libraries, including ones using forward-mode differentiation for when you have very few parameters. Also keep in mind that the loss functions we have defined can be sped up by not either declaring the global variables as `const` or by passing them via so-called *closures*.